# Abnormality Detection in Musculoskeletal Radiographs

In [1]:
from keras.applications.densenet import DenseNet169, DenseNet121, preprocess_input
from keras.preprocessing.image import ImageDataGenerator, load_img, image
from keras.models import Sequential, Model, load_model, Input
from keras.layers import Conv2D, MaxPool2D
from keras.layers import Activation, Dropout, Flatten, Dense, average
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
import pandas as pd
from tqdm import tqdm
import os
import numpy as np
import random
from keras.optimizers import Adam
import keras.backend as K
import cv2
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [16]:
def ensemble(models, model_input):
    # collect outputs of models in a list
    yModels=[model(model_input) for model in models] 
    # averaging outputs
    yAvg=average(yModels) 
    # build model from same input and avg output
    modelEns = Model(inputs=model_input, outputs=yAvg,    name='ensemble')  
   
    return modelEns

In [6]:
models=[]
model_xr_elbow = load_model("densenet_mura_rs_v3_xr_elbow.h5")
model_xr_elbow.name = 'model_xr_elbow'
model_xr_finger = load_model("densenet_mura_rs_v3_xr_finger.h5")
model_xr_finger.name = 'model_xr_finger'
model_xr_forearm = load_model("densenet_mura_rs_v3_xr_forearm.h5")
model_xr_forearm.name = 'model_xr_forearm'
model_xr_hand = load_model("densenet_mura_rs_v3_xr_hand.h5")
model_xr_hand.name = 'model_xr_hand'
model_xr_shoulder = load_model("densenet_mura_rs_v3_xr_shoulder.h5")
model_xr_shoulder.name = 'model_xr_shoulder'
model_xr_wrist = load_model("densenet_mura_rs_v3_xr_wrist.h5")
model_xr_wrist.name = 'model_xr_wrist'
model_xr_humerus = load_model("densenet_mura_rs_v3_xr_humerus.h5")
model_xr_humerus.name = 'model_xr_humerus'

models.append(model_xr_elbow)
models.append(model_xr_finger)
models.append(model_xr_forearm)
models.append(model_xr_hand)
models.append(model_xr_shoulder)
models.append(model_xr_wrist)
models.append(model_xr_humerus)


In [ ]:
model_input = Input(shape=models[0].input_shape[1:]) # c*h*w
modelEns = ensemble(models, model_input)

In [18]:
modelEns.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
model_xr_elbow (Model)          (None, 1)            12644545    input_5[0][0]                    
__________________________________________________________________________________________________
model_xr_finger (Model)         (None, 1)            12644545    input_5[0][0]                    
__________________________________________________________________________________________________
model_xr_forearm (Model)        (None, 1)            12644545    input_5[0][0]                    
__________________________________________________________________________________________________
model_xr_h

In [47]:
modelEns.save("densenet_mura_rs_v3_ensemble.h5")